In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

def train(epoch):
    model.train()
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 50 == 49:
            print(f"Epoch {epoch}, Batch {i+1}, Loss: {running_loss / 50:.3f}")
            running_loss = 0.0

def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    print(f"Test Accuracy: {100. * correct / total:.2f}%")

for epoch in range(1, 100):
    train(epoch)
    test()
    scheduler.step()

torch.save(model.state_dict(), "cifar10-resnet18.ckpt")
print("Model saved as cifar10-resnet18.ckpt")


Using cuda device


100%|██████████| 170M/170M [00:09<00:00, 17.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1, Batch 50, Loss: 3.655
Epoch 1, Batch 100, Loss: 2.297
Epoch 1, Batch 150, Loss: 2.062
Epoch 1, Batch 200, Loss: 1.882
Epoch 1, Batch 250, Loss: 1.759
Epoch 1, Batch 300, Loss: 1.692
Epoch 1, Batch 350, Loss: 1.672
Test Accuracy: 43.44%
Epoch 2, Batch 50, Loss: 1.598
Epoch 2, Batch 100, Loss: 1.576
Epoch 2, Batch 150, Loss: 1.518
Epoch 2, Batch 200, Loss: 1.513
Epoch 2, Batch 250, Loss: 1.494
Epoch 2, Batch 300, Loss: 1.466
Epoch 2, Batch 350, Loss: 1.456
Test Accuracy: 50.13%
Epoch 3, Batch 50, Loss: 1.395
Epoch 3, Batch 100, Loss: 1.377
Epoch 3, Batch 150, Loss: 1.376
Epoch 3, Batch 200, Loss: 1.320
Epoch 3, Batch 250, Loss: 1.329
Epoch 3, Batch 300, Loss: 1.296
Epoch 3, Batch 350, Loss: 1.286
Test Accuracy: 55.92%
Epoch 4, Batch 50, Loss: 1.233
Epoch 4, Batch 100, Loss: 1.234
Epoch 4, Batch 150, Loss: 1.210
Epoch 4, Batch 200, Loss: 1.196
Epoch 4, Batch 250, Loss: 1.171
Epoch 4, Batch 300, Loss: 1.158
Epoch 4, Batch 350, Loss: 1.096
Test Accuracy: 61.54%
Epoch 5, Batch 50, L

In [ ]:
from google.colab import files
files.download("cifar10-resnet18.ckpt")


-rw-r--r-- 1 root root 44806508 Nov 19 20:50 cifar10-resnet18.ckpt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>